In [2]:
import pandas as pd
from src.charts.charts import ExcelWriterWrapper, ChartMaker

### ExcelWriterWrapper
принимает аргументы: путь, куда сохранить файл и словарь формата sheet_name: DataFrame. Для сохранения надо вызвать метод close_writer.
#### Пример ввода

In [31]:
test_df = pd.DataFrame([1, 2, 3])
test_dict = {'sheet1': test_df, 'sheet2': test_df, 'sheet3': test_df}
save_path = 'test.xlsx'
writer = ExcelWriterWrapper(save_path, test_dict)
writer.close_writer()
test_df

,0
0,1
1,2
2,3


#### Результат

In [10]:
df = pd.read_excel('test.xlsx', sheet_name=None, index_col=0)
df = pd.concat([df[sheet] for sheet in df.keys()], axis=1)
df

,0,0,0
0,1,1,1
1,2,2,2
2,3,3,3


### ChartMaker 
Основывается на классе ExcelWriterWrapper. Дополнительно принимает аргументы: charts_list - лист кортежей формата (chart_name, sheet_name, x_axis_name, y_axis_name) и place_chart - лист с названиями ячеек куда сохранять графики.
#### Пример

In [64]:
charts_list = [("Name", f"sheet{i}", f"x_axis{i}", f"y_axis{i}") for i in range(1, 4)]
place_chart = [f"A{i}" for i in range(1, 62, 20)]
chart_maker = ChartMaker(save_path, charts_list, place_chart, test_dict)
print(chart_maker.charts)
print('--'*20)
print(chart_maker.place_charts)

[('Name', 'sheet1', 'x_axis1', 'y_axis1'), ('Name', 'sheet2', 'x_axis2', 'y_axis2'), ('Name', 'sheet3', 'x_axis3', 'y_axis3')]
----------------------------------------
['A1', 'A21', 'A41', 'A61']


Во время итерации по самому себе класс добавляет графики и возращает номер графика, имя и лист, куда сохраниться. Так что достаточно лишь добавить нужные данные на график. 

#### P.S. при добавлении графика, добавляется ложная серия. Надо не забыть ее удалить

In [65]:
for i, name, sheet_name in chart_maker:
    print(f"Номер графика -- {i}, Имя графика -- {name}, Имя листа -- {sheet_name}")
    print("---"*10)
    chart = chart_maker.workbook.charts[i]
    del chart.series[0]
    for i in range(5):
        chart.add_series({'categories': [sheet_name, i, 0, i*i, 0],
                          'values': [sheet_name, i*2, 4, i+i, 8]})
    print(f"Количество графиков -- {len(chart_maker.workbook.charts)}")
    print(f"Количество серий на графике -- {len(chart.series)}")
    print("Пример x и y серии:")
    x = chart.series[3]['categories']
    print(f"categories-X ---> {x}")
    y = chart.series[3]['values']
    print(f"values-Y ---> {y}")

Номер графика -- 0, Имя графика -- Name, Имя листа -- sheet1
------------------------------
Количество графиков -- 1
Количество серий на графике -- 5
Пример x и y серии:
categories-X ---> sheet1!$A$4:$A$10
values-Y ---> sheet1!$E$7:$I$7
Номер графика -- 1, Имя графика -- Name, Имя листа -- sheet2
------------------------------
Количество графиков -- 2
Количество серий на графике -- 5
Пример x и y серии:
categories-X ---> sheet2!$A$4:$A$10
values-Y ---> sheet2!$E$7:$I$7
Номер графика -- 2, Имя графика -- Name, Имя листа -- sheet3
------------------------------
Количество графиков -- 3
Количество серий на графике -- 5
Пример x и y серии:
categories-X ---> sheet3!$A$4:$A$10
values-Y ---> sheet3!$E$7:$I$7


До сохранения можно внести небольшие измения в графики. Кардинальные проще задать изначально в классе. Также надо не забыть, закрыть writer, чтобы сохранить результат

In [63]:
print("Начальное значение --", chart.x_axis['min'])
chart.x_axis['min'] = 2
print("Измененное значение --", chart.x_axis['min'])
chart_maker.close_writer()

Начальное значение -- None
Измененное значение -- 2
